European Central Bank (ECB) API with Python
=====

## Data on Eurozone countries

-----

*February 2026*<br>
*@bd_econ*

ECB API Documentation is [here](https://data.ecb.europa.eu/help/api/overview)

This example requests data from the ECB's SDMX API using Python's `requests` library with CSV output. The ECB provides data on interest rates, exchange rates, monetary aggregates, and more.

You can browse available datasets using the [ECB Data Portal](https://data.ecb.europa.eu/).

In [ ]:
import requests
import pandas as pd
import io

## Parameters/Settings

Browse datasets on the [ECB Data Portal](https://data.ecb.europa.eu/). The API base URL is:

```
https://data-api.ecb.europa.eu/service/data/{dataflow}/{key}
```

The `key` is a dot-separated string of dimension values. Use `+` to request multiple values for the same dimension.

### Example 1: Government Bond Yields

Monthly 10-year government bond yields for selected Eurozone countries, since 2012.

In [ ]:
# List of Eurozone countries
c_list = ['DE', 'FR', 'IT', 'ES', 'PT', 'GR', 'NL', 'BE', 'AT', 'IE']

# IRS = Interest Rate Statistics
# Key: Freq.Country.Type.TransType.Maturity.Sector.Currency.BusCov.FVType
key = f"M.{'+'.join(c_list)}.L.L40.CI.0000.EUR.N.Z"

url = f'https://data-api.ecb.europa.eu/service/data/IRS/{key}'
params = {'startPeriod': '2012', 'detail': 'dataonly'}
headers = {'Accept': 'text/csv'}

r = requests.get(url, params=params, headers=headers)
print(f'Status: {r.status_code}')

## Pandas DataFrame

The CSV response has columns including `REF_AREA`, `TIME_PERIOD`, and `OBS_VALUE`. Pivot to get countries as columns.

In [ ]:
df = pd.read_csv(io.StringIO(r.text))
data = df.set_index(['TIME_PERIOD', 'REF_AREA'])['OBS_VALUE'].unstack()
data.index = pd.to_datetime(data.index)
data.index.name = ''
data.columns.name = ''
data.tail()

In [ ]:
%matplotlib inline
data[['DE', 'FR', 'IT', 'ES', 'GR']].plot(
    title='10-Year Government Bond Yields, in percent',
    figsize=(12, 5));

### Example 2: EUR/USD Exchange Rate

Monthly EUR/USD exchange rate from the EXR (Exchange Rates) dataflow.

In [ ]:
url = 'https://data-api.ecb.europa.eu/service/data/EXR/M.USD.EUR.SP00.A'
params = {'startPeriod': '2015', 'detail': 'dataonly'}
headers = {'Accept': 'text/csv'}

r = requests.get(url, params=params, headers=headers)
df = pd.read_csv(io.StringIO(r.text))
data = df.set_index('TIME_PERIOD')['OBS_VALUE']
data.index = pd.to_datetime(data.index)
data.index.name = ''
data.plot(title='EUR/USD Exchange Rate', legend=False);

### Example 3: Unemployment Rate (AMECO)

Annual unemployment rates from the AMECO macroeconomic database.

Note: the AMECO dataflow uses 3-letter country codes (DEU, FRA, etc.) and names its country column `AME_REF_AREA` rather than `REF_AREA`.

In [ ]:
countries = 'DEU+FRA+ITA+ESP+PRT+GRC+NLD+BEL+AUT+IRL'
key = f'A.{countries}.1.0.0.0.ZUTN'

url = f'https://data-api.ecb.europa.eu/service/data/AME/{key}'
params = {'startPeriod': '2015', 'detail': 'dataonly'}
headers = {'Accept': 'text/csv'}

r = requests.get(url, params=params, headers=headers)
df = pd.read_csv(io.StringIO(r.text))
data = df.set_index(['TIME_PERIOD', 'AME_REF_AREA'])['OBS_VALUE'].unstack()
data.index.name = ''
data.columns.name = ''
data.tail()